In [1]:
import torch
import numpy as np
import os
from torch.utils.data import Dataset, DataLoader
import torch.utils.data as utils

In [2]:
DATA_DIR_DEEPTHOUGHT="/storage/yw18581/data"
data_dir = DATA_DIR_DEEPTHOUGHT
train_test = os.path.join(data_dir, "train_validation_test")

In [3]:
data = np.load("/storage/yw18581/data/train_validation_test/Xy_train+val_clean_300_24_10_25.npz")
x = data["x"]
y = data['y']

tensor_x = torch.from_numpy(x) # transform to torch tensors
tensor_y = torch.from_numpy(y)

xy_dataset = utils.TensorDataset(tensor_x,tensor_y)

In [63]:
class UNetDataset(Dataset):
    def __init__(self, xy_dataset, transform=None):
        self.transform = transform
        self._data = xy_dataset

    def __getitem__(self, idx):
        image = self._data[idx][0]
        mask = self._data[idx][1]
        sample = {'image': image, 'masks': mask}

        if self.transform:
            sample = self.transform(sample)
        return sample

    def __len__(self):
        return len(self._data)


class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        image, mask = sample['image'], sample['masks']
        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        image = image.transpose(2,0)
        mask = mask.transpose(2,0)
        return {'image': image,
                'masks': mask}


In [64]:
train_dataset = UNetDataset(xy_dataset, transform=ToTensor())

In [68]:
dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=4)

In [69]:
len(train_dataset)

960

In [70]:
for i, data in enumerate(dataloader):
    print(i, data['image'].shape, data['masks'].shape)
    if i==4:
        break

0 torch.Size([4, 1, 1400, 1400]) torch.Size([4, 1, 1400, 1400])
1 torch.Size([4, 1, 1400, 1400]) torch.Size([4, 1, 1400, 1400])
2 torch.Size([4, 1, 1400, 1400]) torch.Size([4, 1, 1400, 1400])
3 torch.Size([4, 1, 1400, 1400]) torch.Size([4, 1, 1400, 1400])
4 torch.Size([4, 1, 1400, 1400]) torch.Size([4, 1, 1400, 1400])
